In [1]:
from matplotlib import pyplot as plt
from windrose import WindroseAxes
from matplotlib import pyplot as plt
from utils.data import get_data
from utils.util import getTopLevelPath
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from mpl_toolkits.basemap import Basemap
import os, numpy as np, pandas as pd

In [2]:
chunks_path = getTopLevelPath() + 'data/Model/chunks/'
chunk_files = [chunks_path + chunk for chunk in os.listdir(chunks_path) if chunk.endswith('.feather')]

In [ ]:
def mae(x):
    if len(x) > 1:
        MAE = mean_absolute_error(x.f, x.ws_15)
    else:
        MAE = np.nan
    return x.stod.iloc[0], x.lon.iloc[0], x.lat.iloc[0], MAE, len(x)

In [ ]:
def mape(x):
    if len(x) > 1:
        MAPE = mean_absolute_percentage_error(x.f, x.ws_15)
    else:
        MAPE = np.nan
    return x.stod.iloc[0], x.lon.iloc[0], x.lat.iloc[0], MAPE, len(x)

In [ ]:
tqdm.pandas(desc = 'Calculate MAPE for each group')
df = df[df.f > 0]
res = df.groupby('stod').progress_apply(mape)

In [ ]:
bins = [i for i in range(0, 30, 5)] + [np.inf]

In [ ]:
cut_f = pd.cut(df.f, bins = bins).value_counts().sort_index()
cut_ws_15 = pd.cut(df.ws_15, bins = bins).value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots()
width = 0.35  # width of the bars

bins_labels = [f'[{bins[i]}, {bins[i+1]}[' for i in range(len(bins) - 1)]
x = range(len(bins_labels))

ax.bar(x, cut_f, width, label='Measured wind speed')
ax.bar([i + width for i in x], cut_ws_15, width, label='Reanalysis wind speed')

ax.set_xlabel('Wind speed range [m/s]')
ax.set_ylabel('Frequency')
ax.set_title('Frequency of Values by Bins')
ax.set_xticks([i + width / 2 for i in x])
ax.set_xticklabels(bins_labels, rotation=45, ha='right')
ax.legend()
outputpath = getTopLevelPath() + 'data/Visualization/png/freqMeasuredVsReanalysis.png'
plt.tight_layout()
plt.savefig(outputpath)
plt.show()


In [ ]:
# Define coordinates and MAE values (replace with your data)
lons, lats, mape_values = [r[1] for r in res], [r[2] for r in res], [r[3] for r in res]# Replace with actual longitudes

In [ ]:
# Create Basemap instance for Iceland
plt.figure(figsize = (10, 10)) 
m = Basemap(projection='merc', llcrnrlat=63.2, urcrnrlat=66.8, llcrnrlon=-24.7, urcrnrlon=-13.3, resolution='h')

# Draw coastline and boundaries of Iceland
m.drawcoastlines()
m.drawcountries()
m.drawmapboundary()

# Convert latitudes and longitudes to Basemap coordinates
x, y = m(lons, lats)

# Plot MAE values as colored points
m.scatter(x, y, c=mape_values, cmap='OrRd', edgecolors='none', alpha=0.9)

# Add colorbar
plt.colorbar(label='MAPE', fraction = 0.035, pad = 0.04)
plt.tight_layout()
# Show plot
outputpath = getTopLevelPath() + 'data/Visualization/png/MAPEoverIceland.png'
plt.savefig(outputpath)
plt.show()


In [ ]:
data = get_data()

In [ ]:
data = data.drop([i for i in range(720)], axis = 1)

In [ ]:
def plotAllScatterFVFg(dirpath = './data/parts/'):
    files = [dirpath + file for file in os.listdir(dirpath) if file.endswith('.feather')]

    for file in files:
        data = pd.read_feather(file)
        data = data[data.f < data.fg]
        data = data[data.fg <= 75]
        plt.scatter(data.f, data.fg, s = 0.1, c = 'grey', marker = 'x')

    plt.xlabel('f')
    plt.ylabel('fg')
    plt.show()

In [ ]:
# Plot wind rose
ax = WindroseAxes.from_ax()
ax.bar(data.d, data.f, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title('Wind Rose Plot')
plt.show()

In [ ]:
import seaborn as sns
correlation_matrix = data[['f', 'ws_15', 'ws_250', 'ws_500']].corr()
# Plot correlation heatmap
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Calculate the difference in wind direction between reanalysis and measured data
wind_direction_difference = data['wd_15'] - data['d']

# Remove potential wrap-around (e.g., if wind direction is 350 degrees and 10 degrees)
wind_direction_difference = np.where(wind_direction_difference > 180, wind_direction_difference - 360, wind_direction_difference)
wind_direction_difference = np.where(wind_direction_difference < -180, wind_direction_difference + 360, wind_direction_difference)

wind_direction_difference = abs(wind_direction_difference)

# Plot histogram of wind direction difference
plt.figure(figsize=(8, 6))
sns.histplot(wind_direction_difference, bins=40, kde=True, color='skyblue', edgecolor='black')
plt.xlabel('Munur á vindátt')
plt.ylabel('Fjöldi')
#plt.savefig('./data/figures/wind_direction_difference.png')
plt.show()


In [ ]:
# Plot histograms of wind speed from measurement and reanalysis data
plt.figure(figsize=(10, 6))
diff_15 = abs(data.f - data.ws_15)
diff_250 = abs(data.f - data.ws_250)
diff_500 = abs(data.f - data.ws_500)
sns.histplot(diff_15, bins=20, kde=True, color='blue', label='Difference for 15 meters')
sns.histplot(diff_250, bins=20, kde=True, color='green', label='Difference for 250 meters')
sns.histplot(diff_500, bins=20, kde=True, color='yellow', label='Difference for 500 meters')
plt.xlabel('Wind speed (m/s)')
plt.ylabel('Frequency')
plt.legend()
#plt.savefig('./data/figures/wind_speed_difference_each_en.png')
plt.show()

In [ ]:
# Plot histograms of wind speed from measurement and reanalysis data
plt.figure(figsize=(10, 6))
sns.histplot(data.f, bins=20, kde=True, color='blue', label='Measured at 10 meters')
sns.histplot(data.ws_15, bins=20, kde=True, color='orange', label='Reanalysis at 15 meters')
plt.xlabel('Wind speed (m/s)')
plt.ylabel('Frequency')
plt.legend()
#plt.savefig('./data/figures/wind_speed_difference_en.png')
plt.show()

In [ ]:
data[data.stod == 1350]

In [ ]:
abDiff = np.abs(data.f - data.ws_15)
skew = 3*(np.mean(abDiff) - np.median(abDiff))/np.std(abDiff)

skew

In [ ]:
# Plot histograms of wind speed from measurement and reanalysis data
plt.figure(figsize=(10, 6))
sns.histplot(data[data.stod == 1350].f, bins=20, kde=True, color='blue', label='Measured at 10 meters')
sns.histplot(data[data.stod == 1350].ws_15, bins=20, kde=True, color='orange', label='Reanalysis at 15 meters')
plt.xlabel('Wind speed (m/s)')
plt.ylabel('Frequency')
plt.legend()
#plt.savefig('./data/figures/wind_speed_difference_en.png')
plt.show()